<a id="1"></a> <br>
# <center><span style="font-family:cursive;">If you liked my work then please upvote, Thank you.</span></center>

# Contents

* [Import Libraries and the Data](#1)
* [Descriptive Analysis](#2)
    - [Basic Descriptions of the Data and Features](#21)
    - [Pandas Profiling](#22)
    - [Handling Missing Values](#23)
    - [Feature Engineering](#24)
    - [Data Selection](#25)
* [Sales Data Analysis](#3)
    - [How much did we sell in each month?](#31)
    - [How much and how many of each product did we sell each month?](#32)
    - [How many of each product did we sell each month?](#33)
    - [How much did we sell in each city? (Bar Chart)](#34)
    - [How much did we sell in each state? (Interactive Geographical Map)](#35)
    - [How much of each product did we sell in each city?](#36)
    - [How many of each product did we sell in each city?](#37)
    - [Do we have different customers in the same state? If yes, how much and how many has each of them ordered?](#38)
    - [When are the rush hours?](#39)
    - [Association rules-apriori algorithm](#40)

<a id="1"></a> <br>
# <center><span style="font-family:cursive;">Import Libraries and the Data</span></center>

In [ ]:
import pandas as pd
import numpy as np
import os
from pandas_profiling import ProfileReport
from matplotlib import pyplot as plt
import seaborn as sns
import folium
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
for i in os.listdir('../input/sales-product-data'):
    print(i)

In [ ]:
# This part of the could is retrieved from KNIGHTBEARR's notebook (https://www.kaggle.com/knightbearr/sales-data-deep-analysis-knightbearr)

# let's make a list compreension for all the data in the folder
files = [file for file in os.listdir('../input/sales-product-data')]

# let's make a pandas DataFrame
all_months_data = pd.DataFrame()

# makes a loop for concat the data
for file in files:
    data = pd.read_csv("../input/sales-product-data/" + file)
    all_months_data = pd.concat([all_months_data, data])

<a id="2"></a> <br>
# <center><span style="font-family:cursive;">Descriptive Analysis</span></center>

<a id="21"></a> <br>
## Basic Descriptions of the Data and Features

In [ ]:
all_months_data.shape

In [ ]:
all_months_data.tail()

In [ ]:
all_months_data.head()

In [ ]:
all_months_data.info()

In [ ]:
all_months_data.columns

In [ ]:
all_months_data[['Order ID', 'Product', 'Order Date', 'Purchase Address']].describe()

In [ ]:
all_months_data['Quantity Ordered'] = pd.to_numeric(all_months_data['Quantity Ordered'], errors = 'coerce')
all_months_data['Price Each'] = pd.to_numeric(all_months_data['Price Each'], errors = 'coerce')

In [ ]:
all_months_data.describe()

<a id="22"></a> <br>
## Pandas Profiling

We can also benefit from using Pandas Profiling as well. It is an amazing tool that can generate thorough profile reports and interesting plots for EDA and descriptive analysis purposes.

In [ ]:
profile = ProfileReport(all_months_data)
profile

<a id="23"></a> <br>
## Handling Missing Values

In [ ]:
all_months_data.isna().sum()

In [ ]:
all_months_data[all_months_data["Order ID"].isna()]

In [ ]:
all_months_data = all_months_data.dropna()
all_months_data.shape

In [ ]:
all_months_data.isna().sum()

<a id="24"></a> <br>
## Feature Engineering

In [ ]:
# Sale Related
all_months_data['Sale'] = all_months_data['Quantity Ordered']*all_months_data['Price Each']

In [ ]:
# Address Related
all_months_data['City'] = all_months_data['Purchase Address'].apply(lambda x: x.split(',')[1])
all_months_data['State'] = all_months_data['Purchase Address'].apply(lambda x: x.split()[-2])
all_months_data['Postal Code'] = all_months_data['Purchase Address'].apply(lambda x: x.split()[-1])

In [ ]:
# Time Related
all_months_data['Order Date'] = pd.to_datetime(all_months_data['Order Date'])

all_months_data['Year'] = all_months_data['Order Date'].dt.year
all_months_data['Month'] = all_months_data['Order Date'].dt.month
all_months_data['Hour'] = all_months_data['Order Date'].dt.hour

In [ ]:
all_months_data.head(10)

<a id="25"></a> <br>
## Data Selection

In [ ]:
print(all_months_data['Year'].value_counts())

Since a limited number of data is recorded in 2020, we only choose 2019 data for our analysis.

In [ ]:
all_months_data = all_months_data.drop(all_months_data[all_months_data['Year']==2020].index)

In [ ]:
all_months_data.shape

<a id="3"></a> <br>
# <center><span style="font-family:cursive;">Sales Data Analysis</span></center>

In each following subsection here, I will propose a question and answer it along with some plots and analysis. So, to make this section clearer, I assumed that "how much" product refers to *Total Sale* and "how many" refers to *Quantity Ordered*. The idea of using *Total Sale* along with *Quantity Ordered* is that *Total Sale* may not be a suitable and sufficient measure to only take a look at. Maybee price of a product would be so high that it could influence the total sale of that month but, it is ordered rarely. Moreover, taking a look at the *Quantity Ordered* would provide us with more useful information that we can use in inventory management and logistics.

<a id="31"></a> <br> 
## How much and how many products did we sell in each month?

In [ ]:
# data used in this section
temp_data = all_months_data.groupby(['Month']).sum().reset_index()

#
fig, axes = plt.subplots(2, 1, figsize = (40,25))
fig.subplots_adjust(hspace=.3)

sns.barplot(x='Month', y='Sale', data=temp_data, ax=axes[0])
axes[0].set_xlabel(axes[0].get_xlabel(), size=30)
axes[0].set_ylabel(axes[0].get_ylabel(), size=30)
axes[0].set_xticklabels(axes[0].get_xticklabels(), size=30)
axes[0].bar_label(axes[0].containers[0], fmt='%.2f', size=25)
axes[0].set_title('Total Sales per Month', size= 40)

# -------

# 
sns.barplot(x='Month', y='Quantity Ordered', data=temp_data, ax=axes[1])
axes[1].set_xlabel(axes[1].get_xlabel(), size=30)
axes[1].set_ylabel(axes[1].get_ylabel(), size=30)
axes[1].set_xticklabels(axes[1].get_xticklabels(), size=30)
axes[1].bar_label(axes[1].containers[0], fmt='%.2f', size=25)
axes[1].set_title('Total Quantity Ordered per Month', size= 40)

In [ ]:
'''''
# data used in this section
temp_data = all_months_data.groupby(['Month', 'Product']).sum().reset_index()

#
fig, axes = plt.subplots(2, 1, figsize = (40,25))
sns.barplot(x='Month', y='Sale', hue='Product', data=temp_data, ax=axes[0])


#
axes[0].set_xlabel(axes[0].get_xlabel(), size=30)
axes[0].set_ylabel(axes[0].get_ylabel(), size=30)

#
axes[0].set_xticklabels(axes[0].get_xticklabels(), size=30)

#
# -------
#
sns.barplot(x='Month', y='Quantity Ordered', hue='Product', data=temp_data, ax=axes[1])

#
axes[1].set_xlabel(axes[1].get_xlabel(), size=30)
axes[1].set_ylabel(axes[1].get_ylabel(), size=30)

#
axes[1].set_xticklabels(axes[1].get_xticklabels(), size=30)
'''''

<a id="32"></a> <br> 
## How much did each product sell each month?

By using FacetGrid here, we can have different subplots with the same scale in the axes. Each subplot here refers to a specific month that is listed as a title above the subplot. *Y*-axis shows the products, and *X*-axis displays "how much" or "how many" of that product we have sold in that specific month. Since all axes have the same scale, now, it is easier to compare the *Total Sale* and *Quantity ordered* of different products over months.

In [ ]:
# data used in this section
temp_data = all_months_data.groupby(['Month', 'Product']).sum().reset_index()

#
g = sns.FacetGrid(temp_data, col="Month", hue='Month', col_wrap=4, size=8)
g.map(sns.barplot, "Quantity Ordered", "Product")
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('Title', fontsize=36)

<a id="33"></a> <br> 
## How many did each product sell each month?

In [ ]:
# data used in this section
temp_data = all_months_data.groupby(['Month', 'Product']).sum().reset_index()

#
ax = sns.FacetGrid(temp_data, col="Month", hue='Month', col_wrap=4, size=8)
ax.map(sns.barplot, "Sale", "Product")
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('Title', fontsize=36)

<a id="34"></a> <br> 
## How much and how many did we sell in each city? (Bar Chart)

In [ ]:
# data used in this section
temp_data = all_months_data.groupby(['City']).sum().reset_index()

#
fig, axes = plt.subplots(2, 1, figsize = (50, 30))
sns.barplot(x='City', y='Sale', data=temp_data, ax=axes[0])
axes[0].set_xlabel(axes[0].get_xlabel(), size=30)
axes[0].set_ylabel(axes[0].get_ylabel(), size=30)
axes[0].set_xticklabels(axes[0].get_xticklabels(), size=30)
axes[0].bar_label(axes[0].containers[0], fmt='%.2f', size=25)
axes[0].set_title('h', size= 40)

# -------

# 
sns.barplot(x='City', y='Quantity Ordered', data=temp_data, ax=axes[1])
axes[1].set_xlabel(axes[1].get_xlabel(), size=30)
axes[1].set_ylabel(axes[1].get_ylabel(), size=30)
axes[1].set_xticklabels(axes[1].get_xticklabels(), size=30)
axes[1].bar_label(axes[1].containers[0], fmt='%.2f', size=25)
axes[1].set_title('h', size= 40)

<a id="35"></a> <br> 
## How much and how many did we sell in each city? (Interactive Geographical Map )

An Interactive Geographical Map is an interesting tool to plot geospatial data. So, here we have a US state map, and a statistical variable, in this case, sale, is shown on the map. The intensity of the color,  which is basically the brightness and dullness of the color, shows us how much we have sold in each state. The color intensity scale is shown at the top of the map, and as it is clear, we have sold more in California.

In [ ]:
temp_data = all_months_data.groupby(['State']).sum().reset_index()[['State', 'Sale']]

us_map = folium.Map(location=[40, -95], zoom_start=4)

url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=temp_data,
    columns=["State", "Sale"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="Sale ($)",
).add_to(us_map)

folium.LayerControl().add_to(us_map)

us_map

<a id="36"></a> <br> 
## How much of each product did we sell in each city?

For a deeper analysis, we can also take a look at the "Sale" and the "Quantity Ordered" of each product in each city as well.

In [ ]:
# data used in this section
temp_data = all_months_data.groupby(['City', 'Product']).sum().reset_index()

ax = sns.FacetGrid(temp_data, col="City", hue='City', col_wrap=3, size=8)
ax.map(sns.barplot, "Sale", "Product")
ax.fig.subplots_adjust(top=0.9)
ax.fig.suptitle('City/Product/Total Sale', fontsize=36)

<a id="37"></a> <br> 
## How many of each product did we sell in each city?

In [ ]:
# data used in this section
temp_data = all_months_data.groupby(['City', 'Product']).sum().reset_index()

ax = sns.FacetGrid(temp_data, col="City", hue='City', col_wrap=3, size=8)
ax.map(sns.barplot, "Quantity Ordered", "Product")
ax.fig.subplots_adjust(top=0.9)
ax.fig.suptitle('City/Product/Quantity Ordered', fontsize=36)

<a id="38"></a> <br> 
## Do we have different customers in the same state? If yes, how much and how many has each of them ordered?

It might be beneficial to take a look at the different customers in the same state or even in the same city, which is not the case here. It can give us a good picture of our customers in a state and maybe leads us further to personalized marketing.

In [ ]:
all_months_data[['Postal Code', 'State', 'City']].value_counts()

As it is clear, we have two different customers in California. Let's look at how much and how many they have ordered in 2019.

In [ ]:
# data used in this section
temp_data = all_months_data.groupby(['Postal Code', 'State', 'City']).sum().reset_index()
temp_data.head(11).sort_values(by=['Sale'], ascending=False)[['Postal Code', 'State', 'City', 'Sale', 'Quantity Ordered']]

As the table represents, the customer in San Fransisco has ordered more than the Los Angeles one. Now, to make an interesting note to keep in mind, we could also take a look at what products they have ordered more for a deeper analysis. In that way, we can see which products are more popular in which states/cities. For this analysis plz take a look at this section, [How many of each product did we sell in each city?](#37).

<a id="39"></a> <br> 
## When are the rush hours?

In [ ]:
# data used in this section
temp_data = pd.concat([all_months_data.groupby(['Hour']).count()['Product'], 
                       all_months_data.groupby(['Hour']).sum()[['Sale', 'Quantity Ordered']]], axis=1).reset_index()
temp_data.columns = ['Hour', 'Number of Orders', 'Total Sale', 'Quantity Ordered']

temp_data

You can find a thorough rush hour analysis in the below plot based on the Quantity Ordered, Number of Orders, and Total Sales in each hour.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize = (50, 50))

axes[0].plot(temp_data['Hour'], temp_data['Quantity Ordered'], '-p', color='blue', markerfacecolor='blue', 
             markersize=20, linewidth=4, label = "Quantity Ordered")
axes[0].plot(temp_data['Hour'], temp_data['Number of Orders'], '-p', color='red', markerfacecolor='red', 
             markersize=20, linewidth=4, label = "Number of Orders")
axes[0].legend(fontsize=25)

axes[1].plot(temp_data['Hour'], temp_data['Total Sale'], '-p', color='black', markerfacecolor='black', 
             markersize=20, linewidth=4, label = "Total Sale")
axes[1].legend(fontsize=25)

<a id="40"></a> <br> 
## Association rules-apriori algorithm

There're lots of great articles and videos about what the apriori algorithm is. So, I won't go through the theory behind it but, in a nutshell, it is an analysis that at last, tells us sentences like these:

- "If *this* then *that*" 
- "People who did *that* also did *that*" 
- "People who watched *that* also watched *that*" 
- "People who bought *that* also bought *that*" 

One of the greatest stories in data science is the legend beer-and-diapers story. The legend says that (Full story [here](https://canworksmart.com/diapers-beer-retail-predictive-analytics/#:~:text=The%20legend%20says%20that%20a,have%20beer%20in%20their%20carts).):

> A study was done by a retail grocery store. The findings were that men between 30-40 years in age, shopping between 5 pm and 7 pm on Fridays, who purchased **diapers** were most likely to also have **beer** in their carts.

Besides the fact that why and how buying diapers and beers should be correlated, this sentence is exactly what kind of information we can extract from the apriori algorithm or in a wider and broader terminology, association rules.

As I mentioned, I won't go through the theory behind this algorithm but, there're three terminologies needed to be discussed:

1. Support 
2. Confidence
3. Lift

Consider a Market Basket Optimization problem. Support in such a problem gives us an idea of how frequent a product or an item is in all the transactions. It has the following formula for product $1$ ($P1$):

> $$Support(P1) = \frac{\text{# of transactions containing P1}}{\text{Total # of transactions}}$$

In the same kind of problem, Confidence defines as how likely a product or an item is in the transaction given that the transaction already has a second product (or products). It has the following formula for product $1$ ($P1$) and product $2$ ($P2$):

> $$Confidence(P1->P2) = \frac{\text{# of transactions containing P1 and P2}}{\text{# of transactions containing P1}}$$

So in a simpler form, it says how often $P2$ appears in transactions that contain $P1$ only.


Finally, lift tells us how likely $P2$ is purchased when $P1$ is purchased. In another term, how much our confidence is lifted that $P2$ will be purchased given that $P1$ is already purchased. It has the following formula:

> $$Lift(P1) = \frac{Confidence(P1->P2)}{Support(P1)}$$

A lift value above $1$ means that $P2$ is likely to be bought if $P1$ is bought. Usually, a good value for lift is above $2$. Yet, I didn't use the same procedure for this problem in this notebook, and I only took a look at the supports.

To implement the apriori algorithm in Python, I've seen people using the *mlxtend* library in Kaggle but, I found the *Apyori* library a better option since it provides the confidence and the lift values as well. **Moreover, in association rules mining, the left-hand side (lhs) and right-hand side (rhs) of the rules are not interchangeable. Meaning that: $lhs -> rhs$ is not equal to $rhs -> lhs$.** Therefore, we have to specify those in the rules too.

You can download and install this package [here](https://github.com/ymoch/apyori).

In [ ]:
# This line of code is just added here to enable us to import this library into a notebook on Kaggle.  
# If you want to use this package in your notebook on Kaggle, you have to first download the whole package from the GitHub address that was provided above.
# Then, you should add it as input data in your notebook and run the below code.

!cp -r ../input/aphorism/apyori-master/* ./ 
from apyori import apriori

In order to use such an algorithm, we need to have a list of transactions like this:
> [[USB-C Charging Cable], [iPhone, Apple Airpods Headphones], [Wired Headphones, iPhone]]

To build such a list, we can use the groupby method in pandas. 

In [ ]:
grouped_by_address_and_product = all_months_data.groupby(by = 'Purchase Address')['Product'].apply(lambda x:x.tolist())
grouped_by_address_and_product = grouped_by_address_and_product.reset_index()
grouped_by_address_and_product.columns = ['Addr', 'Products']

grouped_by_address_and_product['Len'] = grouped_by_address_and_product['Products'].apply(lambda x: len(x))

As I looked through the transactions, I realized that many transactions have only one product in their baskets. This could affect the results of our analysis (take a look at the support formula). Therefore, I considered two types of setups for the apriori algorithm here. 

* First, I derived the association rules for all transactions without minding how many items are in a basket. 
* Second, I only chose those transactions that had more than 1 item in their baskets. 

By doing so, we could also take a look at the results of the two setups, and see their differences too.

In [ ]:
transactions_no_length = grouped_by_address_and_product['Products']
transactions_with_length = grouped_by_address_and_product[grouped_by_address_and_product['Len']>1]['Products']

print(transactions_no_length.head())
print('\n')
print(transactions_with_length.head())

In [ ]:
transactions = transactions_no_length.tolist()
print(type(transactions))

results_no_length = list(apriori(transactions=transactions, min_support=0.0001))
results_no_length[10:15]

In [ ]:
transactions = transactions_with_length.tolist()
print(type(transactions))

results_with_length = list(apriori(transactions=transactions, min_support=0.0001))
results_with_length[10:15]

The results of the apriori algorithm using the *Apyori* library may be somehow unclear. The following code will make a clear table of the results. Note that I was only interested in the rules with more than one item in them. That's why I added that if condition in the function. If you are interested in all of the rules, you have to change that part of the code.

In [ ]:
def inspect(results):
    lhs = []
    rhs = []
    supports = []
    confidences = []
    lifts = []
    
    for result in results:
        if len(tuple(result[2][0][1]))>1:
            lhs.append(tuple(result[2][1][0])[0])
            rhs.append(tuple(result[2][1][1])[0])
            supports.append(result[1])
            confidences.append(result[2][1][2])
            lifts.append(result[2][1][3])
            
    return pd.DataFrame({'lhs': lhs,
                         'rhs': rhs,
                         'supports': supports,
                         'confidences': confidences,
                         'lifts': lifts})

In [ ]:
results_inspected=inspect(results_no_length)
results_inspected[results_inspected['supports']>0.008]

In [ ]:
results_inspected=inspect(results_with_length)
results_inspected[results_inspected['supports']>0.03]

An interesting point here is that most of the rules that we found in of the setups could be found in the other but, it has different support, lift, and confidence values which we expected since we are using more transactions in one of the setups. Now, consider the first setup and the first found rule. The information that this rule gives us is something like these statements:

> In 0.8% of transactions, AA Batteries (4-pack) and AAA Batteries (4-pack) are bought together.

> If there are AA Batteries (4-pack) in a transaction, there is a 5% probability that this customer will buy AAA Batteries (4-pack) too.

> the lift value of 0.42 will tell us that these two items are not much dependent on each other and are unlikely to be bought together.

We can have the same interpretation for the second setup as well. However, the assumption in that setup is that **we are certain that there is more than one item in the basket**.

<a id="1"></a> <br>
# <center><span style="font-family:cursive;">If you liked my work then please upvote, Thank you.</span></center>